In [ ]:
!pip install concepcy

In [ ]:
import spacy
import concepcy

nlp = spacy.load('en_core_web_sm')
# Using default concepCy configuration
nlp.add_pipe('concepcy')

In [ ]:
doc = nlp('Walter Alvarez discovered evidence of dinosaur extinction near Gubbio, Italy.')

# Access all the 'RelatedTo' relations from the Doc
for word, relations in doc._.relatedto.items():
    print(f'Word: {word} {relations}')

# Access the 'RelatedTo' relations word by word
for token in doc:
    print(f'Word: {token} {token._.relatedto}')

In [ ]:
!pip install networkx

In [ ]:
!pip install matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict

# Process the text
doc = nlp('Walter Alvarez discovered evidence of dinosaur extinction near Gubbio, Italy.')

# Create a NetworkX graph
G = nx.Graph()

# Dictionary to store all relations for better organization
all_relations = defaultdict(list)

# Collect all relations
for word, relations in doc._.relatedto.items():
    print(f'Word: {word} -> {relations}')
    
    # Add the main word as a node
    G.add_node(word, node_type='source')
    
    # Add related concepts as nodes and create edges
    for related_concept in relations:
        # Handle case where related_concept might be a dict
        if isinstance(related_concept, dict):
            # Extract the concept name/label from the dictionary
            concept_name = related_concept.get('label', str(related_concept))
            concept_id = related_concept.get('id', concept_name)
            
            # Use the concept name as the node identifier
            G.add_node(concept_name, 
                      node_type='related',
                      concept_id=concept_id,
                      full_data=str(related_concept))
            G.add_edge(word, concept_name, relation='RelatedTo')
            all_relations[word].append(concept_name)
            
            print(f"  -> {concept_name} (from: {related_concept})")
        else:
            # Handle case where it's already a string
            G.add_node(related_concept, node_type='related')
            G.add_edge(word, related_concept, relation='RelatedTo')
            all_relations[word].append(related_concept)

print(f"\nGraph info:")
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

# Create the visualization
plt.figure(figsize=(15, 10))

# Define node positions using spring layout for better visualization
pos = nx.spring_layout(G, k=2, iterations=50)

# Separate source nodes (original words) from related nodes for different styling
source_nodes = [node for node, data in G.nodes(data=True) if data.get('node_type') == 'source']
related_nodes = [node for node, data in G.nodes(data=True) if data.get('node_type') == 'related']

# Draw the network
# Draw source nodes (original words from the text)
nx.draw_networkx_nodes(G, pos, nodelist=source_nodes, 
                      node_color='lightblue', node_size=1500, alpha=0.8)

# Draw related nodes (ConceptNet concepts)
nx.draw_networkx_nodes(G, pos, nodelist=related_nodes, 
                      node_color='lightcoral', node_size=800, alpha=0.6)

# Draw edges
nx.draw_networkx_edges(G, pos, alpha=0.5, edge_color='gray', width=1)

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')

plt.title("ConceptNet 'RelatedTo' Relations Network\nBlue: Original words, Red: Related concepts", 
          fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

# Print detailed analysis
print("\n" + "="*50)
print("DETAILED ANALYSIS")
print("="*50)

for word in source_nodes:
    related_concepts = list(G.neighbors(word))
    print(f"\n'{word}' is related to:")
    for i, concept in enumerate(related_concepts, 1):
        print(f"  {i}. {concept}")

# Optional: Save the graph
# plt.savefig('conceptnet_relations.png', dpi=300, bbox_inches='tight')

# Optional: Create an interactive plot using NetworkX and matplotlib
# You can also export to other formats like GEXF for use in Gephi
# nx.write_gexf(G, "conceptnet_relations.gexf")